## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/Sample.csv"
file_type = "csv"

# CSV options
infer_schema = "true" #For getting the correct data type of the columns
first_row_is_header = "true" #First row will be header
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


display(df)

ID,Name,Age
1,Sagar,20
2,Shivam,23
3,Muni,26


In [0]:
df.show()

+---+------+---+
| ID|  Name|Age|
+---+------+---+
|  1| Sagar| 20|
|  2|Shivam| 23|
|  3|  Muni| 26|
+---+------+---+



In [0]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)



In [0]:
#1st Method
df.filter(df.ID==1).show()

+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|  1|Sagar| 20|
+---+-----+---+



In [0]:
#2nd Method
from pyspark.sql.functions import col
df.filter(col("ID")==1).show()

+---+-----+---+
| ID| Name|Age|
+---+-----+---+
|  1|Sagar| 20|
+---+-----+---+



In [0]:
df.filter((col("ID")==1) | (col("Name")=="Shivam")).show()

+---+------+---+
| ID|  Name|Age|
+---+------+---+
|  1| Sagar| 20|
|  2|Shivam| 23|
+---+------+---+



In [0]:
df.filter((col("ID")==1) & (col("Name")=="Shivam")).show()

+---+----+---+
| ID|Name|Age|
+---+----+---+
+---+----+---+



In [0]:
df.filter(col("ID")!=1).show()

+---+------+---+
| ID|  Name|Age|
+---+------+---+
|  2|Shivam| 23|
|  3|  Muni| 26|
+---+------+---+



In [0]:
# 1st method
df1 = df.withColumnRenamed("ID","ID_New")
df1.show()

+------+------+---+
|ID_New|  Name|Age|
+------+------+---+
|     1| Sagar| 20|
|     2|Shivam| 23|
|     3|  Muni| 26|
+------+------+---+



In [0]:
df1 = df.withColumnRenamed("ID","ID_New").withColumnRenamed("Name","Name_New").withColumnRenamed("Age","Age_New")
df1.show()

+------+--------+-------+
|ID_New|Name_New|Age_New|
+------+--------+-------+
|     1|   Sagar|     20|
|     2|  Shivam|     23|
|     3|    Muni|     26|
+------+--------+-------+



In [0]:
# 2nd Method of renaming column
df2 = df.selectExpr("ID as ID_New","Name as Name_New","Age as Age_New")
df2.show()

+------+--------+-------+
|ID_New|Name_New|Age_New|
+------+--------+-------+
|     1|   Sagar|     20|
|     2|  Shivam|     23|
|     3|    Muni|     26|
+------+--------+-------+



In [0]:
#3rd Method of renaming column
from pyspark.sql.functions import col

df3 = df.select(col("ID").alias("ID_New"),col("Name").alias("Name_New"),col("Age").alias("Age_New"))
df3.show()

+------+--------+-------+
|ID_New|Name_New|Age_New|
+------+--------+-------+
|     1|   Sagar|     20|
|     2|  Shivam|     23|
|     3|    Muni|     26|
+------+--------+-------+



In [0]:
# 1st method
from pyspark.sql.functions import lit
df4 = df.withColumn("Country",lit("India"))
df4.show()

+---+------+---+-------+
| ID|  Name|Age|Country|
+---+------+---+-------+
|  1| Sagar| 20|  India|
|  2|Shivam| 23|  India|
|  3|  Muni| 26|  India|
+---+------+---+-------+



In [0]:
#2nd method using existing column
df5 = df.withColumn("Salary",col("ID")*col("Age")*100).withColumn("Country",lit("India"))
df5.show()

+---+------+---+------+-------+
| ID|  Name|Age|Salary|Country|
+---+------+---+------+-------+
|  1| Sagar| 20|  2000|  India|
|  2|Shivam| 23|  4600|  India|
|  3|  Muni| 26|  7800|  India|
+---+------+---+------+-------+



In [0]:
#3rd method
df6 = df.select(col("ID"),col("Name"),col("Age"),lit("India").alias("Country"),lit(col("ID")*col("Age")*100).alias("Salary"))
df6.show()

+---+------+---+-------+------+
| ID|  Name|Age|Country|Salary|
+---+------+---+-------+------+
|  1| Sagar| 20|  India|  2000|
|  2|Shivam| 23|  India|  4600|
|  3|  Muni| 26|  India|  7800|
+---+------+---+-------+------+



In [0]:
# Create a view or table

temp_table_name = "Sample_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from Sample_csv

ID,Name,Age
1,Sagar,20
2,Shivam,23
3,Muni,26


In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "Sample_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)